<a href="https://colab.research.google.com/github/tc-wandering/titanic-survival-prediction/blob/main/titanic_survival_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

These libraries will let you load and explore data (pandas, numpy), visualize it (matplotlib, seaborn), and build models (scikit-learn).com. Next, load the Titanic data (for example, download titanic.csv from Kaggle and upload it to Colab). Read it into a DataFrame

You can quickly inspect it with df.head(), df.info(), and df.describe() to see column names, types, missing values, and summary stats. Perform exploratory data analysis (EDA): plot histograms or count plots of features like Pclass, Sex, and Age to see patterns. For example, you might see women and children had higher survival rates. Use df.isnull().sum() to find missing values.

In [ ]:
!pip install pandas numpy matplotlib seaborn scikit-learn
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix

df = pd.read_csv("/content/train.csv")

display(df.head())
display(df.info())
display(df.describe())
display(df.isnull().sum())



,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


None

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


,0
PassengerId,0
Survived,0
Pclass,0
Name,0
Sex,0
Age,177
SibSp,0
Parch,0
Ticket,0
Fare,0


Data cleaning: Following common practice, fill missing ages with the median age, fill missing Embarked values with the mode, and drop columns that won’t help prediction (e.g. Cabin has many missing values)

Next, drop unneeded columns like PassengerId, Name, and Ticket as they are identifiers not useful for predicting survival

In [ ]:
df = pd.read_csv("/content/train.csv")

# Fill missing ages with the median, using the recommended syntax
df['Age'] = df['Age'].fillna(df['Age'].median())

# Fill missing Embarked values with the mode, using the recommended syntax
df['Embarked'] = df['Embarked'].fillna(df['Embarked'].mode()[0])

# Drop columns that won't help prediction
df.drop(columns=['Cabin'], inplace=True)
df.drop(columns=['PassengerId','Name','Ticket'], inplace=True)

display(df.head())

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


Feature encoding: Convert categorical fields to numeric form. For example, encode Sex and Embarked using a label encoder or one-hot encoding.

In [ ]:
# Feature Encoding
le = LabelEncoder()

# Encode 'Sex' column
df['Sex'] = le.fit_transform(df['Sex'])

# Encode 'Embarked' column
df['Embarked'] = le.fit_transform(df['Embarked'])

display(df.head())

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,2
1,1,1,0,38.0,1,0,71.2833,0
2,1,3,0,26.0,0,0,7.9250,2
3,1,1,0,35.0,1,0,53.1000,2
4,0,3,1,35.0,0,0,8.0500,2


Now split the data: separate features X and target y (Survived), then use train_test_split

This holds 20% of data for testing

In [ ]:
from sklearn.preprocessing import StandardScaler

X = df.drop('Survived', axis=1)
y = df['Survived']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42)

Modeling: Try simple classifiers and compare:
1. Logistic Regression
2. Random Forest
3. Support Vector Machine (SVM)

In [ ]:
from sklearn.linear_model import LogisticRegression
log_model = LogisticRegression(max_iter=1000)
log_model.fit(X_train, y_train)
y_pred_log = log_model.predict(X_test)
print("Logistic Regression Accuracy:", accuracy_score(y_test, y_pred_log))

from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)
print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_rf))

from sklearn.svm import SVC
svm_model = SVC()
svm_model.fit(X_train, y_train)
y_pred_svm = svm_model.predict(X_test)
print("SVM Accuracy:", accuracy_score(y_test, y_pred_svm))

print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_rf))

Logistic Regression Accuracy: 0.8044692737430168
Random Forest Accuracy: 0.8156424581005587
SVM Accuracy: 0.8156424581005587
Confusion Matrix:
 [[91 14]
 [19 55]]


# Task
Hard code a complete machine learning project from scratch, without using any predefined functions from libraries like scikit-learn or pandas. The project should include the following steps:

1.  **Data Loading and Preprocessing:**
    *   Load the dataset from `"train.csv"`.
    *   Handle missing values: fill 'Age' with the median and 'Embarked' with the mode.
    *   Remove the 'Cabin', 'PassengerId', 'Name', and 'Ticket' columns.
    *   Encode categorical features ('Sex' and 'Embarked') into numerical values.
    *   Split the data into training and testing sets.
    *   Standardize the features.

2.  **Model Implementation:**
    *   Implement Logistic Regression from scratch.
    *   Implement a Random Forest classifier from scratch.
    *   Implement a Support Vector Machine (SVM) classifier from scratch.

3.  **Training and Evaluation:**
    *   Train each of the implemented models on the preprocessed training data.
    *   Evaluate the accuracy of each model on the test set using a custom-coded accuracy function.

## Load and parse csv

### Subtask:
Load and parse csv.


**Reasoning**:
Read the CSV file, store the header and data, and print the first few rows.



In [ ]:
import csv

with open('/content/train.csv', 'r') as file:
    reader = csv.reader(file)
    header = next(reader)
    data = list(reader)

print("Header:", header)
print("First 5 rows of data:", data[:5])

Header: ['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']
First 5 rows of data: [['1', '0', '3', 'Braund, Mr. Owen Harris', 'male', '22', '1', '0', 'A/5 21171', '7.25', '', 'S'], ['2', '1', '1', 'Cumings, Mrs. John Bradley (Florence Briggs Thayer)', 'female', '38', '1', '0', 'PC 17599', '71.2833', 'C85', 'C'], ['3', '1', '3', 'Heikkinen, Miss. Laina', 'female', '26', '0', '0', 'STON/O2. 3101282', '7.925', '', 'S'], ['4', '1', '1', 'Futrelle, Mrs. Jacques Heath (Lily May Peel)', 'female', '35', '1', '0', '113803', '53.1', 'C123', 'S'], ['5', '0', '3', 'Allen, Mr. William Henry', 'male', '35', '0', '0', '373450', '8.05', '', 'S']]


## Data cleaning and preprocessing

### Subtask:
Handle missing values, similar to `fillna`. This will involve calculating the median for the 'Age' column and the mode for the 'Embarked' column. I will also remove the 'Cabin', 'PassengerId', 'Name', and 'Ticket' columns.


**Reasoning**:
To begin, I will identify the indices of the relevant columns ('Age', 'Embarked', 'Cabin', 'PassengerId', 'Name', 'Ticket') from the `header` list. Then, I will extract the non-empty 'Age' and 'Embarked' values, calculate the median age and the mode for the embarked location, and fill in the missing values in the `data` list accordingly. Finally, I will remove the unnecessary columns and print the first 5 rows of the processed data to verify the changes. This single code block will complete all the steps outlined in the instructions.



In [ ]:
age_index = header.index('Age')
embarked_index = header.index('Embarked')
cabin_index = header.index('Cabin')
passenger_id_index = header.index('PassengerId')
name_index = header.index('Name')
ticket_index = header.index('Ticket')

ages = [float(row[age_index]) for row in data if row[age_index]]
ages.sort()
if len(ages) % 2 == 0:
    mid1 = ages[len(ages) // 2]
    mid2 = ages[len(ages) // 2 - 1]
    median_age = (mid1 + mid2) / 2
else:
    median_age = ages[len(ages) // 2]

embarked_values = [row[embarked_index] for row in data if row[embarked_index]]
embarked_counts = {}
for value in embarked_values:
    embarked_counts[value] = embarked_counts.get(value, 0) + 1
embarked_mode = max(embarked_counts, key=embarked_counts.get)

for row in data:
    if not row[age_index]:
        row[age_index] = median_age
    if not row[embarked_index]:
        row[embarked_index] = embarked_mode

indices_to_remove = {cabin_index, passenger_id_index, name_index, ticket_index}
indices_to_keep = [i for i, _ in enumerate(header) if i not in indices_to_remove]

new_header = [header[i] for i in indices_to_keep]
new_data = [[row[i] for i in indices_to_keep] for row in data]

header = new_header
data = new_data

for row in data[:5]:
    print(row)

['0', '3', 'male', '22', '1', '0', '7.25', 'S']
['1', '1', 'female', '38', '1', '0', '71.2833', 'C']
['1', '3', 'female', '26', '0', '0', '7.925', 'S']
['1', '1', 'female', '35', '1', '0', '53.1', 'S']
['0', '3', 'male', '35', '0', '0', '8.05', 'S']


## Feature encoding

### Subtask:
Write a function to perform label encoding on the 'Sex' and 'Embarked' columns. This will convert the categorical string values into numerical representations.


**Reasoning**:
I will now implement the label encoding for the 'Sex' and 'Embarked' columns. I will first find their indices in the header, then create the mappings and apply them to the data. Finally, I'll print the first 5 rows to verify the changes.



In [ ]:
sex_index = header.index('Sex')
embarked_index = header.index('Embarked')

sex_mapping = {'male': 1, 'female': 0}
embarked_mapping = {'S': 2, 'C': 0, 'Q': 1}

for row in data:
    row[sex_index] = sex_mapping[row[sex_index]]
    row[embarked_index] = embarked_mapping[row[embarked_index]]

for row in data[:5]:
    print(row)

['0', '3', 1, '22', '1', '0', '7.25', 2]
['1', '1', 0, '38', '1', '0', '71.2833', 0]
['1', '3', 0, '26', '0', '0', '7.925', 2]
['1', '1', 0, '35', '1', '0', '53.1', 2]
['0', '3', 1, '35', '0', '0', '8.05', 2]


## Data splitting

### Subtask:
Create a function to split the data into training and testing sets, replacing `train_test_split`. This will involve shuffling the data and then splitting it based on a specified test size.


**Reasoning**:
I need to split the data into training and testing sets. To do this, I will first import the `random` module to shuffle the data, then define the test set size, calculate the split point, and finally separate the data into features (X) and the target variable (y) for both the training and testing sets, printing the shapes to verify.



In [ ]:
import random

random.shuffle(data)

test_size = 0.2
split_index = int(len(data) * (1 - test_size))

train_data = data[:split_index]
test_data = data[split_index:]

X_train = [row[1:] for row in train_data]
y_train = [row[0] for row in train_data]
X_test = [row[1:] for row in test_data]
y_test = [row[0] for row in test_data]

print("X_train shape:", (len(X_train), len(X_train[0])))
print("X_test shape:", (len(X_test), len(X_test[0])))
print("y_train shape:", (len(y_train),))
print("y_test shape:", (len(y_test),))

X_train shape: (712, 7)
X_test shape: (179, 7)
y_train shape: (712,)
y_test shape: (179,)


## Data scaling

### Subtask:
Implement a function to perform standardization on the feature set, replacing `StandardScaler`. This will scale the data to have a mean of 0 and a standard deviation of 1.


**Reasoning**:
I will now implement the feature standardization. I will calculate the mean and standard deviation for each feature in the training set, and then apply this scaling to both the training and test sets. Finally, I will print the first few rows of the scaled data to verify the results.



In [ ]:
import math

def calculate_mean_std(data):
    means = []
    stds = []
    for i in range(len(data[0])):
        column = [float(row[i]) for row in data]
        mean = sum(column) / len(column)
        std = math.sqrt(sum([(x - mean) ** 2 for x in column]) / len(column))
        means.append(mean)
        stds.append(std)
    return means, stds

def standardize_data(data, means, stds):
    scaled_data = []
    for row in data:
        scaled_row = []
        for i, value in enumerate(row):
            scaled_value = (float(value) - means[i]) / stds[i] if stds[i] != 0 else 0
            scaled_row.append(scaled_value)
        scaled_data.append(scaled_row)
    return scaled_data

means, stds = calculate_mean_std(X_train)

X_train_scaled = standardize_data(X_train, means, stds)
X_test_scaled = standardize_data(X_test, means, stds)

print("First 5 rows of X_train_scaled:")
for row in X_train_scaled[:5]:
    print([f"{val:.2f}" for val in row])

print("\nFirst 5 rows of X_test_scaled:")
for row in X_test_scaled[:5]:
    print([f"{val:.2f}" for val in row])


First 5 rows of X_train_scaled:
['0.81', '-1.34', '-2.04', '2.24', '0.78', '-0.23', '0.61']
['-1.58', '0.74', '-0.11', '-0.48', '-0.49', '0.04', '0.61']
['0.81', '-1.34', '-0.11', '-0.48', '-0.49', '-0.48', '-0.64']
['-0.38', '0.74', '1.90', '-0.48', '-0.49', '-0.36', '0.61']
['-1.58', '0.74', '2.21', '-0.48', '-0.49', '-0.07', '-1.89']

First 5 rows of X_test_scaled:
['0.81', '0.74', '-1.19', '3.14', '0.78', '0.12', '0.61']
['0.81', '-1.34', '-0.26', '0.42', '-0.49', '-0.32', '0.61']
['0.81', '0.74', '-0.11', '-0.48', '-0.49', '0.44', '0.61']
['-0.38', '-1.34', '-0.42', '0.42', '-0.49', '-0.14', '0.61']
['-0.38', '0.74', '-0.11', '-0.48', '-0.49', '-0.63', '0.61']


## Logistic regression implementation

### Subtask:
Implement the logistic regression model from scratch. This includes the sigmoid function for predictions, the cost function (binary cross-entropy), and the gradient descent algorithm for training.


**Reasoning**:
I will now implement the logistic regression model from scratch, including the sigmoid function, cost function, gradient descent, and prediction function, as per the user's request.



In [ ]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def cost_function(X, y, weights):
    m = len(y)
    h = sigmoid(np.dot(X, weights))
    cost = -(1/m) * np.sum(y * np.log(h) + (1 - y) * np.log(1 - h))
    return cost

def gradient_descent(X, y, weights, learning_rate, iterations):
    m = len(y)
    for _ in range(iterations):
        predictions = sigmoid(np.dot(X, weights))
        error = predictions - y
        gradient = np.dot(X.T, error) / m
        weights -= learning_rate * gradient
    return weights

def predict(X, weights, threshold=0.5):
    predictions = sigmoid(np.dot(X, weights))
    return [1 if i >= threshold else 0 for i in predictions]

X_train_np = np.array(X_train_scaled)
y_train_np = np.array([float(i) for i in y_train])
X_test_np = np.array(X_test_scaled)
y_test_np = np.array([float(i) for i in y_test])

initial_weights = np.zeros(X_train_np.shape[1])
learning_rate = 0.1
iterations = 1000

trained_weights = gradient_descent(X_train_np, y_train_np, initial_weights, learning_rate, iterations)
y_pred = predict(X_test_np, trained_weights)

correct_predictions = sum(y_pred == y_test_np)
accuracy = correct_predictions / len(y_test_np)

print(f"Logistic Regression Accuracy: {accuracy}")

Logistic Regression Accuracy: 0.7486033519553073


## Random forest implementation

### Subtask:
Implement a Random Forest classifier from scratch. This will involve creating a decision tree implementation first, and then building the random forest on top of that.


**Reasoning**:
I will now implement the Random Forest classifier from scratch, which involves creating a `DecisionTree` class and a `RandomForest` class. The `DecisionTree` class will handle the logic for building a single tree, including Gini impurity calculation and finding the best split. The `RandomForest` class will then create an ensemble of these decision trees, training each on a bootstrapped sample of the data and making predictions based on a majority vote.



In [ ]:
class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

    def is_leaf_node(self):
        return self.value is not None


class DecisionTree:
    def __init__(self, min_samples_split=2, max_depth=100, n_features=None):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.n_features = n_features
        self.root = None

    def fit(self, X, y):
        self.n_features = X.shape[1] if not self.n_features else min(X.shape[1], self.n_features)
        self.root = self._grow_tree(X, y)

    def _grow_tree(self, X, y, depth=0):
        n_samples, n_feats = X.shape
        n_labels = len(np.unique(y))

        if (depth >= self.max_depth or n_labels == 1 or n_samples < self.min_samples_split):
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)

        feat_idxs = np.random.choice(n_feats, self.n_features, replace=False)
        best_feature, best_thresh = self._best_split(X, y, feat_idxs)
        left_idxs, right_idxs = self._split(X[:, best_feature], best_thresh)
        left = self._grow_tree(X[left_idxs, :], y[left_idxs], depth + 1)
        right = self._grow_tree(X[right_idxs, :], y[right_idxs], depth + 1)
        return Node(best_feature, best_thresh, left, right)

    def _best_split(self, X, y, feat_idxs):
        best_gain = -1
        split_idx, split_threshold = None, None

        for feat_idx in feat_idxs:
            X_column = X[:, feat_idx]
            thresholds = np.unique(X_column)

            for thr in thresholds:
                gain = self._information_gain(y, X_column, thr)
                if gain > best_gain:
                    best_gain = gain
                    split_idx = feat_idx
                    split_threshold = thr

        return split_idx, split_threshold

    def _information_gain(self, y, X_column, threshold):
        parent_gini = self._gini(y)
        left_idxs, right_idxs = self._split(X_column, threshold)

        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0

        n = len(y)
        n_l, n_r = len(left_idxs), len(right_idxs)
        gini_l, gini_r = self._gini(y[left_idxs]), self._gini(y[right_idxs])
        child_gini = (n_l / n) * gini_l + (n_r / n) * gini_r
        return parent_gini - child_gini

    def _split(self, X_column, split_thresh):
        left_idxs = np.argwhere(X_column <= split_thresh).flatten()
        right_idxs = np.argwhere(X_column > split_thresh).flatten()
        return left_idxs, right_idxs

    def _gini(self, y):
        hist = np.bincount(y)
        ps = hist / len(y)
        return 1 - np.sum([p ** 2 for p in ps])

    def _most_common_label(self, y):
        counter = np.bincount(y)
        return np.argmax(counter)

    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X])

    def _traverse_tree(self, x, node):
        if node.is_leaf_node():
            return node.value

        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)


class RandomForest:
    def __init__(self, n_trees=10, max_depth=10, min_samples_split=2, n_features=None):
        self.n_trees = n_trees
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.n_features = n_features
        self.trees = []

    def fit(self, X, y):
        self.trees = []
        for _ in range(self.n_trees):
            tree = DecisionTree(
                max_depth=self.max_depth,
                min_samples_split=self.min_samples_split,
                n_features=self.n_features,
            )
            X_sample, y_sample = self._bootstrap_samples(X, y)
            tree.fit(X_sample, y_sample)
            self.trees.append(tree)

    def _bootstrap_samples(self, X, y):
        n_samples = X.shape[0]
        idxs = np.random.choice(n_samples, n_samples, replace=True)
        return X[idxs], y[idxs]

    def _most_common_label(self, y):
        counter = np.bincount(y.astype('int'))
        return np.argmax(counter)

    def predict(self, X):
        predictions = np.array([tree.predict(X) for tree in self.trees])
        tree_preds = np.swapaxes(predictions, 0, 1)
        return np.array([self._most_common_label(pred) for pred in tree_preds])


X_train_rf = np.array(X_train_scaled)
y_train_rf = np.array([int(i) for i in y_train])
X_test_rf = np.array(X_test_scaled)
y_test_rf = np.array([int(i) for i in y_test])

rf = RandomForest(n_trees=10)
rf.fit(X_train_rf, y_train_rf)
y_pred_rf = rf.predict(X_test_rf)

correct_predictions = sum(y_pred_rf == y_test_rf)
accuracy = correct_predictions / len(y_test_rf)
print(f"Random Forest Accuracy: {accuracy}")

ValueError: attempt to get argmax of an empty sequence

**Reasoning**:
The error "ValueError: attempt to get argmax of an empty sequence" in `_most_common_label` indicates that the function is being called with an empty `y` array. This happens when a split results in a node with no samples. I'll add a check in `_grow_tree` to handle cases where a split results in an empty child node, which will prevent this error. I will also fix a bug in `_gini` where an empty array for `y` can cause an error.



In [ ]:
class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

    def is_leaf_node(self):
        return self.value is not None


class DecisionTree:
    def __init__(self, min_samples_split=2, max_depth=100, n_features=None):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.n_features = n_features
        self.root = None

    def fit(self, X, y):
        self.n_features = X.shape[1] if not self.n_features else min(X.shape[1], self.n_features)
        self.root = self._grow_tree(X, y)

    def _grow_tree(self, X, y, depth=0):
        n_samples, n_feats = X.shape
        if n_samples == 0:
            return None
        n_labels = len(np.unique(y))

        if (depth >= self.max_depth or n_labels == 1 or n_samples < self.min_samples_split):
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)

        feat_idxs = np.random.choice(n_feats, self.n_features, replace=False)
        best_feature, best_thresh = self._best_split(X, y, feat_idxs)

        if best_feature is None:
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)

        left_idxs, right_idxs = self._split(X[:, best_feature], best_thresh)
        left = self._grow_tree(X[left_idxs, :], y[left_idxs], depth + 1)
        right = self._grow_tree(X[right_idxs, :], y[right_idxs], depth + 1)
        return Node(best_feature, best_thresh, left, right)

    def _best_split(self, X, y, feat_idxs):
        best_gain = -1
        split_idx, split_threshold = None, None

        for feat_idx in feat_idxs:
            X_column = X[:, feat_idx]
            thresholds = np.unique(X_column)

            for thr in thresholds:
                gain = self._information_gain(y, X_column, thr)
                if gain > best_gain:
                    best_gain = gain
                    split_idx = feat_idx
                    split_threshold = thr
        return split_idx, split_threshold

    def _information_gain(self, y, X_column, threshold):
        parent_gini = self._gini(y)
        left_idxs, right_idxs = self._split(X_column, threshold)

        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0

        n = len(y)
        n_l, n_r = len(left_idxs), len(right_idxs)
        gini_l, gini_r = self._gini(y[left_idxs]), self._gini(y[right_idxs])
        child_gini = (n_l / n) * gini_l + (n_r / n) * gini_r
        return parent_gini - child_gini

    def _split(self, X_column, split_thresh):
        left_idxs = np.argwhere(X_column <= split_thresh).flatten()
        right_idxs = np.argwhere(X_column > split_thresh).flatten()
        return left_idxs, right_idxs

    def _gini(self, y):
        if len(y) == 0:
            return 0
        hist = np.bincount(y)
        ps = hist / len(y)
        return 1 - np.sum([p ** 2 for p in ps])

    def _most_common_label(self, y):
        if len(y) == 0:
            return None
        counter = np.bincount(y)
        return np.argmax(counter)

    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X])

    def _traverse_tree(self, x, node):
        if node is None or node.is_leaf_node():
            return node.value if node else None

        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)


class RandomForest:
    def __init__(self, n_trees=10, max_depth=10, min_samples_split=2, n_features=None):
        self.n_trees = n_trees
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.n_features = n_features
        self.trees = []

    def fit(self, X, y):
        self.trees = []
        for _ in range(self.n_trees):
            tree = DecisionTree(
                max_depth=self.max_depth,
                min_samples_split=self.min_samples_split,
                n_features=self.n_features,
            )
            X_sample, y_sample = self._bootstrap_samples(X, y)
            tree.fit(X_sample, y_sample)
            self.trees.append(tree)

    def _bootstrap_samples(self, X, y):
        n_samples = X.shape[0]
        idxs = np.random.choice(n_samples, n_samples, replace=True)
        return X[idxs], y[idxs]

    def _most_common_label(self, y):
        y = [i for i in y if i is not None]
        if not y:
            return 0
        counter = np.bincount(np.array(y).astype('int'))
        return np.argmax(counter)

    def predict(self, X):
        predictions = np.array([tree.predict(X) for tree in self.trees])
        tree_preds = np.swapaxes(predictions, 0, 1)
        return np.array([self._most_common_label(pred) for pred in tree_preds])


X_train_rf = np.array(X_train_scaled)
y_train_rf = np.array([int(i) for i in y_train])
X_test_rf = np.array(X_test_scaled)
y_test_rf = np.array([int(i) for i in y__test])

rf = RandomForest(n_trees=10)
rf.fit(X_train_rf, y_train_rf)
y_pred_rf = rf.predict(X_test_rf)

correct_predictions = sum(y_pred_rf == y_test_rf)
accuracy = correct_predictions / len(y_test_rf)
print(f"Random Forest Accuracy: {accuracy}")

NameError: name 'y__test' is not defined

**Reasoning**:
I made a typo in the previous code block, using `y__test` instead of `y_test`. I will correct this typo and rerun the code to train and evaluate the Random Forest model.



In [ ]:
class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

    def is_leaf_node(self):
        return self.value is not None


class DecisionTree:
    def __init__(self, min_samples_split=2, max_depth=100, n_features=None):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.n_features = n_features
        self.root = None

    def fit(self, X, y):
        self.n_features = X.shape[1] if not self.n_features else min(X.shape[1], self.n_features)
        self.root = self._grow_tree(X, y)

    def _grow_tree(self, X, y, depth=0):
        n_samples, n_feats = X.shape
        if n_samples == 0:
            return None
        n_labels = len(np.unique(y))

        if (depth >= self.max_depth or n_labels == 1 or n_samples < self.min_samples_split):
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)

        feat_idxs = np.random.choice(n_feats, self.n_features, replace=False)
        best_feature, best_thresh = self._best_split(X, y, feat_idxs)

        if best_feature is None:
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)

        left_idxs, right_idxs = self._split(X[:, best_feature], best_thresh)
        left = self._grow_tree(X[left_idxs, :], y[left_idxs], depth + 1)
        right = self._grow_tree(X[right_idxs, :], y[right_idxs], depth + 1)
        return Node(best_feature, best_thresh, left, right)

    def _best_split(self, X, y, feat_idxs):
        best_gain = -1
        split_idx, split_threshold = None, None

        for feat_idx in feat_idxs:
            X_column = X[:, feat_idx]
            thresholds = np.unique(X_column)

            for thr in thresholds:
                gain = self._information_gain(y, X_column, thr)
                if gain > best_gain:
                    best_gain = gain
                    split_idx = feat_idx
                    split_threshold = thr
        return split_idx, split_threshold

    def _information_gain(self, y, X_column, threshold):
        parent_gini = self._gini(y)
        left_idxs, right_idxs = self._split(X_column, threshold)

        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0

        n = len(y)
        n_l, n_r = len(left_idxs), len(right_idxs)
        gini_l, gini_r = self._gini(y[left_idxs]), self._gini(y[right_idxs])
        child_gini = (n_l / n) * gini_l + (n_r / n) * gini_r
        return parent_gini - child_gini

    def _split(self, X_column, split_thresh):
        left_idxs = np.argwhere(X_column <= split_thresh).flatten()
        right_idxs = np.argwhere(X_column > split_thresh).flatten()
        return left_idxs, right_idxs

    def _gini(self, y):
        if len(y) == 0:
            return 0
        hist = np.bincount(y)
        ps = hist / len(y)
        return 1 - np.sum([p ** 2 for p in ps])

    def _most_common_label(self, y):
        if len(y) == 0:
            return None
        counter = np.bincount(y)
        return np.argmax(counter)

    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X])

    def _traverse_tree(self, x, node):
        if node is None or node.is_leaf_node():
            return node.value if node else None

        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)


class RandomForest:
    def __init__(self, n_trees=10, max_depth=10, min_samples_split=2, n_features=None):
        self.n_trees = n_trees
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.n_features = n_features
        self.trees = []

    def fit(self, X, y):
        self.trees = []
        for _ in range(self.n_trees):
            tree = DecisionTree(
                max_depth=self.max_depth,
                min_samples_split=self.min_samples_split,
                n_features=self.n_features,
            )
            X_sample, y_sample = self._bootstrap_samples(X, y)
            tree.fit(X_sample, y_sample)
            self.trees.append(tree)

    def _bootstrap_samples(self, X, y):
        n_samples = X.shape[0]
        idxs = np.random.choice(n_samples, n_samples, replace=True)
        return X[idxs], y[idxs]

    def _most_common_label(self, y):
        y = [i for i in y if i is not None]
        if not y:
            return 0
        counter = np.bincount(np.array(y).astype('int'))
        return np.argmax(counter)

    def predict(self, X):
        predictions = np.array([tree.predict(X) for tree in self.trees])
        tree_preds = np.swapaxes(predictions, 0, 1)
        return np.array([self._most_common_label(pred) for pred in tree_preds])


X_train_rf = np.array(X_train_scaled)
y_train_rf = np.array([int(i) for i in y_train])
X_test_rf = np.array(X_test_scaled)
y_test_rf = np.array([int(i) for i in y_test])

rf = RandomForest(n_trees=10)
rf.fit(X_train_rf, y_train_rf)
y_pred_rf = rf.predict(X_test_rf)

correct_predictions = sum(y_pred_rf == y_test_rf)
accuracy = correct_predictions / len(y_test_rf)
print(f"Random Forest Accuracy: {accuracy}")

Random Forest Accuracy: 0.7932960893854749


## Support vector machine (svm) implementation

### Subtask:
Implement a basic Support Vector Machine classifier from scratch. This will be a more complex implementation, focusing on the core concepts of SVM.


**Reasoning**:
I will now implement the Support Vector Machine (SVM) classifier from scratch. I will define a class `SVM` containing `fit` and `predict` methods. The `fit` method will use subgradient descent to find the optimal weights and bias. I will also convert the scaled data and labels into NumPy arrays as required, then create an instance of the `SVM` class, train it, make predictions, and finally calculate and print the accuracy.



In [ ]:
import numpy as np

class SVM:
    def __init__(self, learning_rate=0.001, lambda_param=0.01, n_iters=1000):
        self.lr = learning_rate
        self.lambda_param = lambda_param
        self.n_iters = n_iters
        self.w = None
        self.b = None

    def fit(self, X, y):
        n_samples, n_features = X.shape
        y_ = np.where(y <= 0, -1, 1)
        self.w = np.zeros(n_features)
        self.b = 0

        for _ in range(self.n_iters):
            for idx, x_i in enumerate(X):
                condition = y_[idx] * (np.dot(x_i, self.w) - self.b) >= 1
                if condition:
                    self.w -= self.lr * (2 * self.lambda_param * self.w)
                else:
                    self.w -= self.lr * (2 * self.lambda_param * self.w - np.dot(x_i, y_[idx]))
                    self.b -= self.lr * y_[idx]

    def predict(self, X):
        approx = np.dot(X, self.w) - self.b
        return np.sign(approx)


X_train_svm = np.array(X_train_scaled)
y_train_svm = np.array([int(i) for i in y_train])
X_test_svm = np.array(X_test_scaled)
y_test_svm = np.array([int(i) for i in y_test])

svm = SVM()
svm.fit(X_train_svm, y_train_svm)
y_pred_svm = svm.predict(X_test_svm)

# Convert predictions from -1, 1 to 0, 1
y_pred_svm = np.where(y_pred_svm <= 0, 0, 1)

correct_predictions = np.sum(y_pred_svm == y_test_svm)
accuracy = correct_predictions / len(y_test_svm)
print(f"SVM Accuracy: {accuracy}")

SVM Accuracy: 0.7318435754189944


## Model training and evaluation

### Subtask:
Train each of the implemented models on the preprocessed data and evaluate their performance using an accuracy score function that I will also write from scratch. This will replace `accuracy_score`.


**Reasoning**:
I will define a function `accuracy_score_from_scratch` that calculates the accuracy of the predictions. Then, I will use this function to evaluate the performance of the three previously trained models (Logistic Regression, Random Forest, and SVM) on their respective test sets. Finally, I will print the accuracy for each model with a descriptive label.



In [ ]:
def accuracy_score_from_scratch(y_true, y_pred):
    correct_predictions = sum(1 for true, pred in zip(y_true, y_pred) if true == pred)
    return correct_predictions / len(y_true) if len(y_true) > 0 else 0

y_test_np_list = y_test_np.tolist()
y_pred_list = y_pred
accuracy_log_scratch = accuracy_score_from_scratch(y_test_np_list, y_pred_list)
print(f"Logistic Regression Accuracy (from scratch): {accuracy_log_scratch}")

y_test_rf_list = y_test_rf.tolist()
y_pred_rf_list = y_pred_rf.tolist()
accuracy_rf_scratch = accuracy_score_from_scratch(y_test_rf_list, y_pred_rf_list)
print(f"Random Forest Accuracy (from scratch): {accuracy_rf_scratch}")

y_test_svm_list = y_test_svm.tolist()
y_pred_svm_list = y_pred_svm.tolist()
accuracy_svm_scratch = accuracy_score_from_scratch(y_test_svm_list, y_pred_svm_list)
print(f"SVM Accuracy (from scratch): {accuracy_svm_scratch}")

Logistic Regression Accuracy (from scratch): 0.7486033519553073
Random Forest Accuracy (from scratch): 0.7932960893854749
SVM Accuracy (from scratch): 0.7318435754189944


## Summary:

### Q&A
**What is the accuracy of each of the models implemented from scratch?**
The accuracy of the models on the test set is as follows:
*   **Logistic Regression:** 74.86%
*   **Random Forest:** 79.33%
*   **Support Vector Machine (SVM):** 73.18%

**Which model performed the best?**
The Random Forest model performed the best, with an accuracy of 79.33%.

### Data Analysis Key Findings
*   The dataset was preprocessed by handling missing values, encoding categorical features, and standardizing the data.
*   Three machine learning models were implemented from scratch: Logistic Regression, Random Forest, and Support Vector Machine.
*   The models were trained on the preprocessed training data and evaluated on the test set.
*   The Random Forest model achieved the highest accuracy of 79.33%, followed by Logistic Regression at 74.86% and SVM at 73.18%.

### Insights or Next Steps
*   Since the Random Forest model performed the best, it would be beneficial to further tune its hyperparameters (e.g., number of trees, max depth) to potentially improve its performance.
*   Given the complexity of implementing machine learning models from scratch, using established libraries like scikit-learn for future projects would be more efficient and less error-prone.
